# Planet Data Collection
Using the Open Exoplanet Catalogue database: https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/

## Data License
Copyright (C) 2012 Hanno Rein

Permission is hereby granted, free of charge, to any person obtaining a copy of this database and associated scripts (the "Database"), to deal in the Database without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Database, and to permit persons to whom the Database is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Database. A reference to the Database shall be included in all scientific publications that make use of the Database.

THE DATABASE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE DATABASE OR THE USE OR OTHER DEALINGS IN THE DATABASE.

## Follow instructions to get the xml file

In [1]:
import xml.etree.ElementTree as ET, urllib.request, gzip, io
url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems.xml.gz"
oec = ET.parse(gzip.GzipFile(fileobj=io.BytesIO(urllib.request.urlopen(url).read())))

## Parse into Pandas DataFrame
Information on what each field means can be found [here](https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/#data-structure).

In [2]:
import pandas as pd

def parse(base):
    db = oec.findall(f".//{base}")
    
    exclude = ['star', 'videolink', 'binary'] if base in ['system', 'binary'] else ['planet']
    
    columns = set([attribute.tag for attribute in db[0] if attribute.tag not in exclude])
    results = pd.DataFrame(columns=columns)

    for entry in db:
        data = {col : entry.findtext(col) for col in columns}
        if base in ['system', 'binary']:
            data['binaries'] = len(entry.findall('.//binary'))
            data['stars'] = len(entry.findall('.//star'))
        if base in ['system', 'star', 'binary']:
            data['planets'] = len(entry.findall('.//planet'))
        results = results.append(data, ignore_index=True)

    return results

### Parse planet data

In [3]:
planets = parse('planet')
planets.head()

,mass,description,periastrontime,semimajoraxis,discoveryyear,list,eccentricity,period,discoverymethod,lastupdate,periastron,name
0,19.4,11 Com b is a brown dwarf-mass companion to th...,2452899.6,1.29,2008,Confirmed planets,0.231,326.03,RV,15/09/20,94.8,11 Com b
1,11.20,11 Ursae Minoris is a star located in the cons...,2452861.04,1.54,2009,Confirmed planets,0.08,516.22,RV,15/09/20,117.63,11 UMi b
2,4.8,14 Andromedae is an evolved star in the conste...,2452861.4,0.83,2008,Confirmed planets,0,185.84,RV,15/09/20,0,14 And b
3,4.975,The star 14 Herculis is only 59 light years aw...,None,2.864,2002,Confirmed planets,0.359,1766,RV,15/09/21,22.230,14 Her b
4,7.679,14 Her c is the second companion in the system...,None,9.037,2006,Controversial,0.184,9886,RV,15/09/21,189.076,14 Her c


### Parse system data

In [4]:
systems = parse('system')
systems.head()

,declination,distance,name,rightascension,binaries,planets,stars
0,+17 47 34,88.9,11 Com,12 20 43,0.0,1.0,1.0
1,+71 49 26.0466,122.1,11 UMi,15 17 05.88899,0.0,1.0,1.0
2,+39 14 10.3092,79.2,14 And,23 31 17.41346,0.0,1.0,1.0
3,+43 49 18,18.1,14 Her,16 10 23,0.0,2.0,1.0
4,+50 31 30.2153,21.146,16 Cygni,19 41 48.95343,2.0,1.0,3.0


### Parse binary data

In [5]:
binaries = parse('binary')
binaries.head()

,positionangle,separation,name,binaries,planets,stars
0,133.30,39.56,16 Cygni,1.0,1.0,3.0
1,209,3.4,16 Cygni AC,0.0,0.0,2.0
2,237.3,12.37,2M0441+2301,1.0,1.0,3.0
3,79.61,0.2323,2M 044145,0.0,0.0,2.0
4,None,None,2M 1938+4603,0.0,1.0,2.0


### Parse star data

In [6]:
stars = parse('star')
stars.head()

,temperature,magV,mass,spectraltype,magJ,radius,magB,magH,magK,metallicity,name,planets
0,4742,4.74,2.7,G8 III,2.943,19,5.74,2.484,2.282,-0.35,11 Com,1.0
1,4340,5.024,1.80,K4III,2.876,24.08,6.415,2.091,1.939,0.04,11 UMi,1.0
2,4813,5.22,2.2,K0III,3.019,11,6.24,2.608,2.331,-0.24,14 And,1.0
3,5311.0,6.67,1.0,K0 V,5.158,0.708,7.57,4.803,4.714,0.43,14 Her,2.0
4,5825,5.95,1.11,G2V,5.09,1.243,6.59,4.72,4.43,0.096,16 Cygni A,0.0


## Save to CSVs

In [7]:
planets.to_csv('data/planets.csv', index=False)
binaries.to_csv('data/binaries.csv', index=False)
stars.to_csv('data/stars.csv', index=False)
systems.to_csv('data/systems.csv', index=False)

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="../../ch_08/anomaly_detection.ipynb">
        <button>&#8592; Chapter 8</button>
    </a>
    </div>
    <div style="float: right;">
        <a href="./planets_ml.ipynb">
            <button>Next Notebook &#8594;</button>
        </a>
    </div>
</div>
<hr>